In [ ]:
import os, pandas as pd
from datetime import date
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']

CLIENT_SECRETS_FILE = 'client_secret.json'
TOKEN_FILE = 'token.json'

creds = None

# Se já existir token salvo, carrega
if os.path.exists(TOKEN_FILE):
    creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)

# Se não houver token ou estiver expirado, faz login
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
        creds = flow.run_local_server(port=0)
    # Salva o token para reutilizar
    with open(TOKEN_FILE, 'w') as token:
        token.write(creds.to_json())

# Agora pode usar as APIs sem relogar
analytics_data = build('analyticsdata', 'v1beta', credentials=creds)
# analytics_admin = build('analyticsadmin', 'v1alpha', credentials=creds)
analytics_admin = build('analyticsadmin', 'v1beta', credentials=creds)
print("✅ Autenticado com sucesso!")

# accounts_response = analytics_admin.accounts().list().execute()

accounts = []
request = analytics_admin.accounts().list()
while request is not None:
    response = request.execute()
    accounts.extend(response.get('accounts', []))
    request = analytics_admin.accounts().list_next(previous_request=request, previous_response=response)

all_properties = []

# for i in accounts_response['accounts']:
for i in accounts:
    account_name = i['name']
    account_display_name = i['displayName']
    properties_response = analytics_admin.properties().list(
        filter=f"parent:{account_name}"
    ).execute()

    props = properties_response.get('properties', [])

    for prop in props:
        property_id = prop['name']
        property_display = prop['displayName']
        all_properties.append({
            'account_name': account_name,
            'account_display': account_display_name,
            'property_id': property_id,
            'property_display': property_display
        })

base_dados = []

start_of_month = date.today().replace(day=1).isoformat()
end_of_month = date.today().isoformat()

for i in all_properties:
    response = analytics_data.properties().runReport(
        property=i['property_id'],
        body={
            "dateRanges": [{"startDate": start_of_month, "endDate": end_of_month}],
            "metrics": [
                {"name": "sessions"},
                {"name": "transactions"},
                {"name": "purchaseRevenue"}
            ]
        }
    ).execute()

    if "rows" not in response:
        # print(f"Nenhum dado para {i['property_display']} ({i['property_id']})")
        base_dados.append({
            'account_name': i['account_name'],
            'account_display': i['account_display'],
            'property_id': i['property_id'],
            'property_display': i['property_display'],
            'sessions': 0,
            'transactions': 0,
            'purchaseRevenue': 0.0,
            'conversion_rate': 0.0
        })
        continue

    # Extrai os valores
    sessions = int(response["rows"][0]["metricValues"][0]["value"])
    transactions = int(response["rows"][0]["metricValues"][1]["value"])
    purchaseRevenue = float(response["rows"][0]["metricValues"][2]["value"])

    # Calcula a taxa de conversão (%)
    conversion_rate = (transactions / sessions) * 100 if sessions > 0 else 0
    base_dados.append({
        'account_name': i['account_name'],
        'account_display': i['account_display'],
        'property_id': i['property_id'],
        'property_display': i['property_display'],
        'sessions': sessions,
        'transactions': transactions,
        'purchaseRevenue': purchaseRevenue,
        'conversion_rate': conversion_rate
    })

print(f"Total de propriedades analisadas: {len(base_dados)}")

df = pd.DataFrame(base_dados)
df.to_csv('relatorio_analytics.csv', index=False, sep=';')


In [ ]:
import pandas as pd, os, streamlit as st
from datetime import date

# ======================
# ⚙️ Configurações gerais
# ======================
st.set_page_config(page_title="Dashboard GA4 – WN7", page_icon="📊", layout="wide")
# logo_path = os.path.join(os.path.dirname(__file__), "assets", "logo.png")

# Pega o diretório onde o script está
base_dir = os.path.dirname(__file__)
logo_path = os.path.join(base_dir, "assents", "logo.png")



# CSS Global — Tema WN7
st.markdown("""
    <style>
        /* ===========================
           🎨 Tema WN7 Performance Digital
        ============================ */
        body, .stApp {
            background-color: #FFFFFF !important;
            color: #1D1D1B !important;
            font-family: 'Montserrat', sans-serif !important;
        }

        h1, h2, h3, h4 {
            color: #005B82 !important;
            font-weight: 600 !important;
        }

        /* Campos de entrada */
        div[data-baseweb="select"], .stTextInput > div > div > input {
            background-color: #FFFFFF !important;
            color: #1D1D1B !important;
            border: 1px solid #ADAFAF !important;
            border-radius: 6px !important;
        }

        /* Placeholders */
        ::placeholder {
            color: #6e6e6e !important;
        }

        /* Botões */
        button[kind="primary"] {
            background-color: #005B82 !important;
            color: white !important;
            border: none !important;
            border-radius: 6px !important;
        }

        button[kind="secondary"] {
            background-color: #F39200 !important;
            color: white !important;
            border: none !important;
            border-radius: 6px !important;
        }

        /* Cards */
        .card {
            background-color: #FFFFFF;
            border: 1px solid #ADAFAF;
            border-radius: 12px;
            padding: 20px;
            margin-bottom: 20px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.05);
            color: #1D1D1B;
        }

        .card h4 {
            color: #005B82;
            margin-bottom: 12px;
            font-size: 26px;
        }

        .receita {
            color: #F39200 !important;
            font-weight: bold;
        }

        /* Linhas e divisores */
        hr {
            border-top: 1px solid #ADAFAF !important;
        }
    </style>
""", unsafe_allow_html=True)

# ======================
# 🖼️ Cabeçalho com logo
# ======================
col_logo, col_titulo = st.columns([0.15, 0.85])
with col_logo:
    st.image(logo_path)
    # st.image("assets/logown7.jpeg", width="stretch")
    # st.image(logo_path, width="stretch")
    # st.image(r"C:\code\agency_dash\agengy\assents\logo.png")

with col_titulo:
    st.title("Dashboard de Contas – Google Analytics 4")
    data_extracao = date.today().strftime("%d/%m/%Y")
    st.markdown(f"**🕒 Dados extraídos em:** {data_extracao}")

st.markdown("---")

# ======================
# 📦 Carrega dados
# ======================
@st.cache_data
def carregar_dados():
    df = pd.read_csv("relatorio_analytics.csv", sep=";")
    df['purchaseRevenue'] = df['purchaseRevenue'].astype(float)
    df['sessions'] = df['sessions'].astype(int)
    df['transactions'] = df['transactions'].astype(int)
    df['conversion_rate'] = df['conversion_rate'].astype(float)
    return df

df = carregar_dados()

# ======================
# 🔹 Identifica contas zeradas e válidas
# ======================
df_zeradas = df[
    (df['sessions'] == 0) &
    (df['transactions'] == 0) &
    (df['purchaseRevenue'] == 0)
]['account_display'].unique()

df_validas = df[
    ~(
        (df['sessions'] == 0) &
        (df['transactions'] == 0) &
        (df['purchaseRevenue'] == 0)
    )
]

# ======================
# 🔍 Filtro de contas
# ======================
contas_disponiveis = sorted(df_validas['account_display'].unique())
selecionadas = st.multiselect(
    "Selecione uma ou mais contas:",
    options=contas_disponiveis,
    placeholder="Escolha as contas que deseja visualizar..."
)

if selecionadas:
    df_filtrado = df_validas[df_validas['account_display'].isin(selecionadas)]
else:
    df_filtrado = df_validas

st.markdown("---")

# ======================
# 🧱 Cards estilizados WN7 (com meta)
# ======================
meta_geral = 100000  # Meta de receita para todas as contas

colunas = st.columns(3)

for idx, conta in enumerate(df_filtrado['account_display'].unique()):
    conta_df = df_filtrado[df_filtrado['account_display'] == conta]

    total_sessions = conta_df['sessions'].sum()
    total_transactions = conta_df['transactions'].sum()
    total_revenue = conta_df['purchaseRevenue'].sum()
    avg_conversion = (total_transactions / total_sessions * 100) if total_sessions > 0 else 0

    # Progresso em relação à meta
    progresso_meta = (total_revenue / meta_geral) * 100
    progresso_meta = min(progresso_meta, 9999)  # Evita porcentagem exagerada

    # Cor do progresso (verde acima da meta, laranja abaixo)
    cor_meta = "#16a34a" if progresso_meta >= 100 else "#F39200"

    col = colunas[idx % 3]
    with col:
        st.markdown(
            f"""
            <div class="card" style="text-align:left;">
                <h4>{conta}</h4>
                <div style="
                    display:grid;
                    grid-template-columns: 1fr 1fr;
                    gap:10px;
                    font-size:17px;
                ">
                    <div><b>Sessões:</b><br>{total_sessions:,}</div>
                    <div><b>Transações:</b><br>{total_transactions:,}</div>
                    <div><b>Receita:</b><br><span class="receita">R$ {total_revenue:,.2f}</span></div>
                    <div><b>Conversão:</b><br>{avg_conversion:.2f}%</div>
                </div>
                <div style="margin-top:10px; font-size:14px;">
                    <span style="color:{cor_meta};"><b>Meta:</b> 100K<br>
                    <b>Progresso:</b> {progresso_meta:.2f}%</span>
                </div>
            </div>
            """,
            unsafe_allow_html=True
        )




# # ======================
# # 🧱 Cards estilizados WN7
# # ======================
# colunas = st.columns(3)

# for idx, conta in enumerate(df_filtrado['account_display'].unique()):
#     conta_df = df_filtrado[df_filtrado['account_display'] == conta]

#     total_sessions = conta_df['sessions'].sum()
#     total_transactions = conta_df['transactions'].sum()
#     total_revenue = conta_df['purchaseRevenue'].sum()
#     avg_conversion = (total_transactions / total_sessions * 100) if total_sessions > 0 else 0

#     col = colunas[idx % 3]
#     with col:
#         st.markdown(
#             f"""
#             <div class="card">
#                 <h4>{conta}</h4>
#                 <div style="
#                     display:grid;
#                     grid-template-columns: 1fr 1fr;
#                     gap:10px;
#                     font-size:17px;
#                 ">
#                     <div><b>Sessões:</b><br>{total_sessions:,}</div>
#                     <div><b>Transações:</b><br>{total_transactions:,}</div>
#                     <div><b>Receita:</b><br><span class="receita">R$ {total_revenue:,.2f}</span></div>
#                     <div><b>Conversão:</b><br>{avg_conversion:.2f}%</div>
#                 </div>
#             </div>
#             """,
#             unsafe_allow_html=True
#         )

# ======================
# ⚠️ Contas zeradas
# ======================
if len(df_zeradas) > 0:
    st.markdown("---")
    st.markdown(
        "<h3 style='text-align:center; color:#F39200;'>⚠️ Contas com todos os valores zerados</h3>",
        unsafe_allow_html=True
    )

    col1, col2, col3, col4 = st.columns(4)
    for idx, conta in enumerate(df_zeradas):
        [col1, col2, col3, col4][idx % 4].markdown(f"- {conta}")


✅ Autenticado com sucesso!
Total de propriedades analisadas: 64
